### 数据读取

In [ ]:
import pandas as pd
import numpy as np
import os
from datasets import Dataset, DatasetDict 
from collections import Counter

file="/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa"
df1=pd.read_csv(file,sep="\t",header=None).rename(columns = {0: "dset", 1: "multi",2:"binary", 3: "seq",4:"Type",5: "detail"})
df = df1.loc[:, ['dset', 'multi','seq']]
df = df[df['dset'] == 'test']

print("+++++++++++++++++++++++++++++Test sets")
print(df.shape)


### label间转换
id2label={"0":"Non-HERV_Coding","1":"HERV_Coding","2":"Non-HERV_Non-Coding","3":"HERV_Non-Coding"}
labels_raw=list(df['multi'])
labels = [id2label[str(i)] for i in labels_raw]

print(Counter(labels))

sequences=list(df['seq'])

In [ ]:
dataset_name="BERT_HERV_Multi_RUN0"
output_dir="/home/u20111010010/Project/DNA-Pretraining/Level1/003.Sequence_Visualization/Dataset_HERV"

### 模型学习到的特征

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 初始化模型和分词器，并将模型移动到GPU上
model_name = "/home/u20111010010/Project/DNA-Pretraining/Level1/002.Model_Classification/Dataset_HERV/Model/BERT_HERV_Multi_RUN0"
model = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 6-mers编码
def kmers(s, k=6):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

def transform_seq(seq):
    return " ".join(kmers(seq)[:300] + kmers(seq)[-212:])


# 特征提取
def extract_features(model, tokenizer, sequences, batch_size=32):
    """
    使用给定的模型和分词器从序列中提取特征，分批进行预测。
    :param model: 使用的模型
    :param tokenizer: 使用的分词器
    :param sequences: 序列列表
    :param batch_size: 每个批次的序列数量
    :return: 所有序列的特征数组
    """
    features = []
    for i in range(0, len(sequences), batch_size):  # 每batch_size个序列进行一次预测
        batch_sequences = sequences[i:i+batch_size]

        # 批量编码
        transformed_reads = [transform_seq(seq) for seq in batch_sequences]
        inputs = tokenizer(transformed_reads, return_tensors="pt", truncation=True, padding=True).to(device)

        outputs = model(**inputs)
        feature = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()
        features.append(feature)
    return np.concatenate(features, axis=0)


### 行数等于输入序列的数量,列数等于模型的last_hidden_state中的特征数量(768)
X = extract_features(model, tokenizer, sequences)
np.save(os.path.join(output_dir, "BERT_HERV_Multi_RUN0_extract_features.npy"), X)

In [ ]:
#X = np.load("/home/u20111010010/Project/DNA-Pretraining/Level1/003.Sequence_Visualization/Dataset_HERV/BERT_HERV_Multi_RUN0_extract_features.npy")
# 自定义title和文件名
title_pca = f"PCA Visualization ({dataset_name})"
title_tsne = f"t-SNE Visualization ({dataset_name})"

### 2D
filename_pca_pdf = os.path.join(output_dir, f"{dataset_name}_pca_visualization")
filename_pca_png = os.path.join(output_dir, f"{dataset_name}_pca_visualization")
filename_tsne_pdf = os.path.join(output_dir, f"{dataset_name}_tsne_visualization")
filename_tsne_png = os.path.join(output_dir, f"{dataset_name}_tsne_visualization")

### 3D
filename_3dpca_pdf = os.path.join(output_dir, f"{dataset_name}_3dpca_visualization.pdf")
filename_3dpca_png = os.path.join(output_dir, f"{dataset_name}_3dpca_visualization.png")
filename_3dtsne_pdf = os.path.join(output_dir, f"{dataset_name}_3dtsne_visualization.pdf")
filename_3dtsne_png = os.path.join(output_dir, f"{dataset_name}_edtsne_visualization.png")

In [ ]:
'''
# PCA降维并可视化(3D)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

#scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=label_colors, cmap='deep', alpha=0.6)
# 对每个唯一标签，画一个散点图，这样我们就可以为每个类别添加一个图例
for label in set(labels):
    idx = np.where(labels == label)
    ax.scatter(X_pca[idx, 0], X_pca[idx, 1], X_pca[idx, 2], label=label, alpha=0.3)

plt.title(title_pca)
ax.legend(loc='best')
plt.savefig(filename_3dpca_pdf)
plt.savefig(filename_3dpca_png)
plt.show()
'''

In [ ]:
'''
# t-SNE降维并可视化(3D)
tsne = TSNE(n_components=10)
X_tsne = tsne.fit_transform(X)
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
#scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=label_colors, cmap='deep', alpha=0.6)
for label in set(labels):
    idx = np.where(labels == label)
    ax.scatter(X_pca[idx, 0], X_pca[idx, 1], X_pca[idx, 2], label=label, alpha=0.3)
    
plt.title(title_tsne)
ax.legend(loc='best')
plt.savefig(filename_3dtsne_pdf)
plt.savefig(filename_3dtsne_png)
plt.show()
'''

In [ ]:
### 字体设置
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.font_manager import FontManager
FontManager()._findfont_cached.cache_clear()

#font_path = "/home/u20111010010/Project/DNA-Pretraining/Level1/Manuscript/Times_New_Roman/Times New Roman.ttf"
font_path ="/home/u20111010010/Project/DNA-Pretraining/Level1/Manuscript/Times_New_Roman/Times_New_Roman.ttf"

#plt.rcParams['font.family'] = 'Times New Roman'
#plt.rcParams['font.serif'] = font_path

import matplotlib.font_manager as fm
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

def HERV_scatterplot(X_pca, labels, title_pca, filename_pca_pdf, filename_pca_png):
    # 颜色设置, 获取 "deep" 调色板的前四个颜色: 深蓝色；橙色；绿色；红色
    colors = sns.color_palette("deep", 4) 
    # 定义标签到颜色的映射
    id2color = {
        'Non-HERV_Coding': colors[0],'HERV_Coding': colors[1],
        'HERV_Non-Coding': colors[2],'Non-HERV_Non-Coding': colors[3]}
    # 定义边框
    plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)
    ###### 创建带图例的图形
    # 绘制散点图
    ax = sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette=id2color, size=0.2, legend='auto')
    # 手动添加图例
    handles, labels_plot = ax.get_legend_handles_labels()
    # 除去最后一个图例项
    plt.legend(handles=handles[:-1], labels=labels_plot[:-1], loc='upper right')
    plt.title(title_pca)
    # 保存图像
    plt.savefig(str(filename_pca_pdf)+".pdf")
    plt.savefig(str(filename_pca_png)+".png",dpi=2000)
    plt.clf()
    ###### 创建不带图例的图形
    # 绘制散点图
    ax = sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette=id2color, size=0.2, legend=False)
    # 手动添加图例
    plt.title(title_pca)
    # 保存图像
    plt.savefig(str(filename_pca_pdf)+"_NonLabel.pdf")
    plt.savefig(str(filename_pca_png)+"_NonLabel.png",dpi=2000)
    # 显示图像
    #plt.show()
    plt.clf()


# 示例调用
# PCA降维并可视化
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)
HERV_scatterplot(X_pca,labels, title_pca, filename_pca_pdf, filename_pca_png)
# t-SNE降维并可视化
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
HERV_scatterplot(X_tsne,labels, title_tsne, filename_tsne_pdf, filename_tsne_png)


In [ ]:
'''
# PCA降维并可视化
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# 颜色设置
# 获取 "deep" 调色板的前四个颜色,深蓝色；橙色；绿色；红色
colors = sns.color_palette("deep", 4) 
# 假设您有一个与序列对应的标签列表 Label1
id2color = {'Non-HERV_Coding': colors[0],'HERV_Coding': colors[1],'HERV_Non-Coding': colors[2],'Non-HERV_Non-Coding': colors[3]}

# 绘制散点图
ax=sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1],hue=labels, palette=id2color,size=0.2,legend='auto')
# 手动添加图例
handles, labels_plot = ax.get_legend_handles_labels()
# 通常情况下，第一个图例项是标题（例如“hue”），然后是hue的不同值，我们只获取这些
plt.legend(handles=handles[:-1], labels=labels_plot[:-1], loc='upper right')
plt.title(title_pca)

plt.savefig(filename_pca_pdf)
plt.savefig(filename_pca_png)
plt.show()
plt.clf()

# t-SNE降维并可视化
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

plt.figure()
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1],hue=labels, palette="deep")
plt.title(title_tsne)
plt.legend(loc='upper right')

plt.savefig(filename_tsne_pdf)
plt.savefig(filename_tsne_png)
plt.show()
'''

### 模型学习特征的无监督新Labels聚类

In [ ]:
### https://ngdc.cncb.ac.cn/hervd/;测试数据156305条序列(83675 HERVs)，3282条HERVs在数据库中有记录（~3.92%）

###### 测试数据中HERV序列
#grep '^test' /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane 'print if ($F[2]==1)'|wc -l
###### 测试数据中与HervD_Atlas数据库重合HERV序列
#bedtools intersect -a  <(perl -le 'while(<>){chomp;next if /^More/ or /Position/ or /(\W+)$/;@F=split/\t+/,$_;@FF=split/\:|\-/,$F[2];print join("\t",@FF),"\t",join("\t",@F[0..$#F-1]);}'  /home/u20111010010/Project/DNA-Pretraining/Level1/000.Data/HervD_Atlas/HERV_Detail.txt)  -b  <(grep '^test' /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";')  -wo|perl -lane 'print "$F[12]\t$F[13]\t$F[14]"'|sort -u|wc -l

###### 交集
#!bedtools intersect -a  <(perl -le 'while(<>){chomp;next if /^More/ or /Position/;@F=split/\t+/,$_;@FF=split/\:|\-/,$F[2];print join("\t",@FF),"\t",join("\t",@F[0..$#F-1]);}'  /home/u20111010010/Project/DNA-Pretraining/Level1/000.Data/HervD_Atlas/HERV_Detail.txt)  -b  <(grep '^test' /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";')  -wo|perl -lane 'print "$F[0]\t$F[1]\t$F[2]\t$F[8]\t",join("\t",@F[-7..-2])' >/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need-Test_NEW_Labels.fa

###### 所有(同一个序列若重叠几种类型，选择重叠片段最长的作为结果;测试数据)
!perl -le 'open(IN,$ARGV[0]);open(IN1,$ARGV[1]);while(<IN>){chomp;@F=split/\t+/,$_;$hash{"$F[0]\t$F[1]\t$F[2]"}="$F[3]\t$F[4]";};while(<IN1>){chomp;@F=split/\t+/,$_;if(exists($hash{"$F[0]\t$F[1]\t$F[2]"})){print $hash{"$F[0]\t$F[1]\t$F[2]"},"\t",join("\t",@F[3..$#F]);}else{if($F[5]==1){print "Unfound\tUnfound\t",join("\t",@F[3..$#F]);}else{print "Non-HERV\tNon-HERV\t",join("\t",@F[3..$#F]);}};}'  <(bedtools intersect -a  <(perl -le 'while(<>){chomp;next if /^More/ or /Position/ or /(\W+)$/;@F=split/\t+/,$_;@FF=split/\:|\-/,$F[2];print join("\t",@FF),"\t",join("\t",@F[0..$#F-1]);}'  /home/u20111010010/Project/DNA-Pretraining/Level1/000.Data/HervD_Atlas/HERV_Detail.txt)  -b  <(grep '^test' /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";')  -wo|perl -lane '@F=split/\t+/,$_;print "$F[12]\t$F[13]\t$F[14]\t$F[8]\t$F[9]\t$F[-1]";'|perl -lane 'BEGIN{%h} @k=($F[0],$F[1],$F[2]); $key=join("-",@k); if(exists $h{$key}){ if($F[4]>$h{$key}[1]){ @{$h{$key}}=@F[3,4]; }}else{ @{$h{$key}}=@F[3,4]; } END{ foreach $k (sort keys %h){ @k=split(/-/, $k); print "$k[0]\t$k[1]\t$k[2]\t$h{$k}[0]\t$h{$k}[1]"; }}')  <(grep '^test' /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";') >/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need-Test_NEW_Labels.fa

###### 所有(同一个序列若重叠几种类型，选择重叠片段最长的作为结果;训练+验证+测试集数据)
!perl -le 'open(IN,$ARGV[0]);open(IN1,$ARGV[1]);while(<IN>){chomp;@F=split/\t+/,$_;$hash{"$F[0]\t$F[1]\t$F[2]"}="$F[3]\t$F[4]";};while(<IN1>){chomp;@F=split/\t+/,$_;if(exists($hash{"$F[0]\t$F[1]\t$F[2]"})){print $hash{"$F[0]\t$F[1]\t$F[2]"},"\t",join("\t",@F[3..$#F]);}else{if($F[5]==1){print "Unfound\tUnfound\t",join("\t",@F[3..$#F]);}else{print "Non-HERV\tNon-HERV\t",join("\t",@F[3..$#F]);}};}'  <(bedtools intersect -a  <(perl -le 'while(<>){chomp;next if /^More/ or /Position/ or /(\W+)$/;@F=split/\t+/,$_;@FF=split/\:|\-/,$F[2];print join("\t",@FF),"\t",join("\t",@F[0..$#F-1]);}'  /home/u20111010010/Project/DNA-Pretraining/Level1/000.Data/HervD_Atlas/HERV_Detail.txt)  -b  <(cat /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";')  -wo|perl -lane '@F=split/\t+/,$_;print "$F[12]\t$F[13]\t$F[14]\t$F[8]\t$F[9]\t$F[-1]";'|perl -lane 'BEGIN{%h} @k=($F[0],$F[1],$F[2]); $key=join("-",@k); if(exists $h{$key}){ if($F[4]>$h{$key}[1]){ @{$h{$key}}=@F[3,4]; }}else{ @{$h{$key}}=@F[3,4]; } END{ foreach $k (sort keys %h){ @k=split(/-/, $k); print "$k[0]\t$k[1]\t$k[2]\t$h{$k}[0]\t$h{$k}[1]"; }}')  <(cat /home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need.fa|perl -lane '@M=split/\:|\-|\_/,$F[-1];print join("\t",@M[0..2]),"\t$_";') >/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need-ALL_NEW_Labels.fa

In [ ]:
file="/home/u20111010010/Project/DNA-Pretraining/Level1/001.Genomics_dataset/Dataset_HERV/VCF_hprc-1000G/Train_Test/data_all_model_HERV-Classification_Need-Test_NEW_Labels.fa"
df1=pd.read_csv(file,sep="\t",header=None).rename(columns = {0:"Class",1:"Group",2: "dset", 3: "multi",4:"binary", 5: "seq",6:"Type",7: "detail"})
df = df1.loc[:, ['dset', 'Class','Group','seq']]

X = np.load("/home/u20111010010/Project/DNA-Pretraining/Level1/003.Sequence_Visualization/Dataset_HERV/BERT_HERV_Multi_RUN0_extract_features.npy")

# 自定义title和文件名
title_pca = f"{dataset_name} PCA Visualization"
title_tsne = f"{dataset_name} t-SNE Visualization"

### 2D
filename_pca_pdf = os.path.join(output_dir, f"{dataset_name}_pca_visualization_NEW_Labels")
filename_pca_png = os.path.join(output_dir, f"{dataset_name}_pca_visualization_NEW_Labels")
filename_tsne_pdf = os.path.join(output_dir, f"{dataset_name}_tsne_visualization_NEW_Labels")
filename_tsne_png = os.path.join(output_dir, f"{dataset_name}_tsne_visualization_NEW_Labels")
### ERV1,ERV2,ERV3,ERVL-MaLR,Gypsy,LTR,Unknown,Unfound

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def HERV_scatterplot(X_pca, labels, title_pca, filename_pca_pdf, filename_pca_png):
    # 设置字体为 "Times New Roman"
    plt.rcParams["font.family"] = "Times New Roman"
    # 颜色设置
    # 获取 "deep" 调色板的前四个颜色: 深蓝色；橙色；绿色；红色
    colors = sns.color_palette("deep", 7) 
    # 定义标签到颜色的映射
    id2color = {'ERV1': colors[0],'ERV2': colors[1],'ERV3': colors[2],'ERVL-MaLR': colors[3],'Gypsy':colors[4],'LTR':colors[5],'Unknown':colors[6],'Unfound':"#A9A9A9",'Non-HERV':"#D3D3D3"}
    # 定义边框
    plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)
    ###### 创建带图例的图形
    ax = sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette=id2color, size=0.2, legend='auto')
    # 手动添加图例
    handles, labels_plot = ax.get_legend_handles_labels()
    # 除去最后一个图例项
    plt.legend(handles=handles[:-1], labels=labels_plot[:-1], loc='upper right')
    plt.title(title_pca)
    # 保存图像
    plt.savefig(str(filename_pca_pdf)+".pdf")
    plt.savefig(str(filename_pca_png)+".png")
    # 显示图像
    plt.show()
    plt.clf()
    ###### 创建不带图例的图形
    ax = sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette=id2color, size=0.2, legend='auto')
    # 手动添加图例
    plt.title(title_pca)
    plt.legend([])
    # 保存图像
    plt.savefig(str(filename_pca_pdf)+"_NonLabel.pdf")
    plt.savefig(str(filename_pca_png)+"_NonLabel.png")
    # 显示图像
    plt.show()
    plt.clf()

In [ ]:
############# 大类绘图

from sklearn.manifold import TSNE
# 示例调用
labels=list(df['Class'])

# t-SNE降维并可视化
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
HERV_scatterplot(X_tsne,labels, title_tsne, filename_tsne_pdf, filename_tsne_png)

In [ ]:
############# 按照Group绘图
labels=list(df['Group'])

filename_pca_pdf = os.path.join(output_dir, f"{dataset_name}_pca_visualization_NEW_Labels-Group")
filename_pca_png = os.path.join(output_dir, f"{dataset_name}_pca_visualization_NEW_Labels-Group")
filename_tsne_pdf = os.path.join(output_dir, f"{dataset_name}_tsne_visualization_NEW_Labels-Group")
filename_tsne_png = os.path.join(output_dir, f"{dataset_name}_tsne_visualization_NEW_Labels-Group")
HERV_scatterplot(X_tsne,labels, title_tsne, filename_tsne_pdf, filename_tsne_png)